In [ ]:
# https://www.youtube.com/watch?v=PvsMg6jFs8E
!pip install langchain
!pip install ctransformers
!pip install sentence-transformers
!pip install faiss-gpu

In [ ]:
# Download the Llama 2 Model:

# llama-2-7b-chat.ggmlv3.q4_0.bin


# From the following link:

# https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGML/tree/main

In [2]:
# Data
# https://www.kaggle.com/datasets/unsdsn/world-happiness?resource=download&select=2019.csv

In [3]:
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import CTransformers
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
import sys

In [4]:
loader = CSVLoader(file_path="2019.csv", encoding="utf-8", csv_args={'delimiter': ','})
data = loader.load()
print(data)

[Document(page_content='Overall rank: 1\nCountry or region: Finland\nScore: 7.769\nGDP per capita: 1.340\nSocial support: 1.587\nHealthy life expectancy: 0.986\nFreedom to make life choices: 0.596\nGenerosity: 0.153\nPerceptions of corruption: 0.393', metadata={'source': '2019.csv', 'row': 0}), Document(page_content='Overall rank: 2\nCountry or region: Denmark\nScore: 7.600\nGDP per capita: 1.383\nSocial support: 1.573\nHealthy life expectancy: 0.996\nFreedom to make life choices: 0.592\nGenerosity: 0.252\nPerceptions of corruption: 0.410', metadata={'source': '2019.csv', 'row': 1}), Document(page_content='Overall rank: 3\nCountry or region: Norway\nScore: 7.554\nGDP per capita: 1.488\nSocial support: 1.582\nHealthy life expectancy: 1.028\nFreedom to make life choices: 0.603\nGenerosity: 0.271\nPerceptions of corruption: 0.341', metadata={'source': '2019.csv', 'row': 2}), Document(page_content='Overall rank: 4\nCountry or region: Iceland\nScore: 7.494\nGDP per capita: 1.380\nSocial sup

In [5]:
# Split the text into Chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
text_chunks = text_splitter.split_documents(data)

print(len(text_chunks))

156


# Embeddings:

In [6]:
import os

# Download Sentence Transformers Embedding From Hugging Face
embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')

# COnverting the text Chunks into embeddings and saving the embeddings into FAISS Knowledge Base
docsearch = FAISS.from_documents(text_chunks, embeddings)


DB_FAISS_PATH = "vectorstore/db_faiss"
if not os.path.isdir(DB_FAISS_PATH):
  os.makedirs(DB_FAISS_PATH)
  docsearch.save_local(DB_FAISS_PATH)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
# Test
query = "What is the value of GDP per capita of Finland provided in the data?"

docs = docsearch.similarity_search(query, k=3)

print("Result", docs)

Result [Document(page_content='Overall rank: 1\nCountry or region: Finland\nScore: 7.769\nGDP per capita: 1.340\nSocial support: 1.587\nHealthy life expectancy: 0.986\nFreedom to make life choices: 0.596\nGenerosity: 0.153\nPerceptions of corruption: 0.393', metadata={'source': '2019.csv', 'row': 0}), Document(page_content='Overall rank: 55\nCountry or region: Estonia\nScore: 5.893\nGDP per capita: 1.237\nSocial support: 1.528\nHealthy life expectancy: 0.874\nFreedom to make life choices: 0.495\nGenerosity: 0.103\nPerceptions of corruption: 0.161', metadata={'source': '2019.csv', 'row': 54}), Document(page_content='Overall rank: 53\nCountry or region: Latvia\nScore: 5.940\nGDP per capita: 1.187\nSocial support: 1.465\nHealthy life expectancy: 0.812\nFreedom to make life choices: 0.264\nGenerosity: 0.075\nPerceptions of corruption: 0.064', metadata={'source': '2019.csv', 'row': 52})]


# Load Quantized Model


In [ ]:
# Downloaded model :

llm = CTransformers(model="llama-2-7b-chat.ggmlv3.q4_0.bin",
                    model_type="llama",
                    max_new_tokens=512,
                    temperature=0.1)

qa_chain = ConversationalRetrievalChain.from_llm(llm, retriever=docsearch.as_retriever())


# Chat bot with history:

In [ ]:
# https://python.langchain.com/docs/use_cases/question_answering/chat_history


while True:
    chat_history = []
    #query = "What is the value of  GDP per capita of Finland provided in the data?"
    query = input(f"Input Prompt: ")
    if query == 'exit':
        print('Exiting')
        sys.exit()
    if query == '':
        continue
    result = qa_chain({"question":query, "chat_history":chat_history})
    print("Response: ", result['answer'])